In [26]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

# Chemin vers le fichier JSONL
file_path = '/home/carolus/Documents/school/green_ia/data/02_openfoodfacts_sample.jsonl'

# Lire le fichier JSONL en utilisant la méthode read_json de pandas avec l'option lines=True
df = pd.read_json(file_path, lines=True)

# Afficher le DataFrame
print(df)


    pnns_groups_1  \
0         unknown   
1         unknown   
2         unknown   
3         unknown   
4         unknown   
..            ...   
995       unknown   
996       unknown   
997       unknown   
998       unknown   
999       unknown   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  ingredients_tags  \
0                                                                                                    

# main preprocessing:

In [27]:
df.rename(columns={'pnns_groups_1': 'groups'}, inplace=True)
df.rename(columns={'ingredients_tags': 'ingredients_temp'}, inplace=True)
df.rename(columns={'product_name': 'name'}, inplace=True)
df.rename(columns={'ecoscore_tags': 'ecoscore_groups'}, inplace=True)
df.rename(columns={'categories_tags': 'categories_temp'}, inplace=True)
df.rename(columns={'ecoscore_score': 'ecoscore_note'}, inplace=True)
df.rename(columns={'labels_tags': 'labels_temp'}, inplace=True)
df.rename(columns={'countries': 'countries_temp'}, inplace=True)


# traitement col GROUPS 
df['groups'] = df['groups'].replace("unknown", None, regex=False)  


# traitement col NAME
df['name'] = df['name'].replace("", None)  
df['name'] = df['name'].replace({np.nan: None})


# traitement col INGREDIENTS
df['ingredients_temp'] = df['ingredients_temp'].replace("", None)  # remplace vide par None
df['ingredients_temp'] = df['ingredients_temp'].replace({np.nan: None}) # remplace NaN par None
df['ingredients_temp'] = df['ingredients_temp'].apply(lambda x: x if isinstance(x, list) else []) # remplace None par liste vide 
df['ingredients_temp'] = df['ingredients_temp'].apply(lambda x: ', '.join(x)) # converti liste en string 

# extraire éléments avec 'en:' nouvelle colonne
def extract_en_ingredients(ingredient_list):
    ingredients = ingredient_list.strip('[]').split(', ')
    return [ingredient.split(':')[-1] for ingredient in ingredients if ingredient.startswith('en:')]
df['ingredients'] = df['ingredients_temp'].apply(extract_en_ingredients)
df.drop(columns=['ingredients_temp'], inplace=True)
df['ingredients'] = df['ingredients'].apply(lambda x: ', '.join(x))
df['ingredients'] = df['ingredients'].replace("", None)  


# traitement col PACKAGING
df['packaging'] = df['packaging'].replace("", None)
def remove_two_letters_and_colon(s):
    if isinstance(s, str):
        return re.sub(r'\b\w{2}:\b', '', s)
    return s
df['packaging'] = df['packaging'].apply(remove_two_letters_and_colon)


# traitement col CODE
df['code'] = df['code'].apply(lambda x: np.nan if pd.isna(x) else int(round(x)))
df['code'] = df['code'].replace("", None)  
df['code'] = df['code'].replace({np.nan: None})


# supprime les lignes où le code ean ou le nom produit sont absents 
df = df[df['name'].notna() & df['code'].notna()]


# traitement col ECOSCORE_GROUPS
df['ecoscore_groups'] = df['ecoscore_groups'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x) # conversion liste vers string 
df['ecoscore_groups'] = df['ecoscore_groups'].replace("unknown", "z")
df['ecoscore_groups'] = df['ecoscore_groups'].replace("", "z")
df['ecoscore_groups'] = df['ecoscore_groups'].fillna("z") 
df['ecoscore_groups'] = df['ecoscore_groups'].replace("not-applicable", "z")


# traitment col CATEGORIES
df['categories_temp'] = df['categories_temp'].replace("", None)  
df['categories_temp'] = df['categories_temp'].replace({np.nan: None}) 
df['categories_temp'] = df['categories_temp'].apply(lambda x: x if isinstance(x, list) else [])
df['categories_temp'] = df['categories_temp'].apply(lambda x: ', '.join(x))

# extraire éléments avec 'en:' nouvelle colonne
def extract_en_categories(ingredient_list):
    ingredients = ingredient_list.strip('[]').split(', ')
    return [ingredient.split(':')[-1] for ingredient in ingredients if ingredient.startswith('en:')]
df['categories'] = df['categories_temp'].apply(extract_en_categories)
df.drop(columns=['categories_temp'], inplace=True)
df['categories'] = df['categories'].apply(lambda x: ', '.join(x))
df['categories'] = df['categories'].replace("", None)  


# traitment col COUNTRIES
df['countries_temp'] = df['countries_temp'].replace("", None)
df['countries_temp'] = df['countries_temp'].replace({np.nan: None})
df['countries_temp'] = df['countries_temp'].apply(lambda x: x if isinstance(x, list) else ([] if x is None else x.split(', ')))

def extract_en_countries(ingredient_list):
    if isinstance(ingredient_list, str):
        ingredient_list = ingredient_list.split(', ')
    return [ingredient.split(':', 1)[-1] for ingredient in ingredient_list if ingredient.startswith('en:')]

df['countries'] = df['countries_temp'].apply(extract_en_countries)
df['countries'] = df['countries'].apply(lambda x: ', '.join(x) if x else None)
df.drop(columns=['countries_temp'], inplace=True)


# traitment col ECOSCORE_NOTE
df['ecoscore_note'] = df['ecoscore_note'].replace("unknown", 999)
df['ecoscore_note'] = df['ecoscore_note'].replace("", 999)
df['ecoscore_note'] = df['ecoscore_note'].fillna(999)


# traitment col LABELS
df['labels_temp'] = df['labels_temp'].replace("", None)
df['labels_temp'] = df['labels_temp'].replace({np.nan: None})
df['labels_temp'] = df['labels_temp'].apply(lambda x: x if isinstance(x, list) else ([] if x is None else x.split(', ')))

def extract_en_countries(ingredient_list):
    if isinstance(ingredient_list, str):
        ingredient_list = ingredient_list.split(', ')
    return [ingredient.split(':', 1)[-1] for ingredient in ingredient_list if ingredient.startswith('en:')]

df['labels'] = df['labels_temp'].apply(extract_en_countries)
df['labels'] = df['labels'].apply(lambda x: ', '.join(x) if x else None)
df.drop(columns=['labels_temp'], inplace=True)

In [28]:
df.head(70)

,groups,packaging,name,ecoscore_groups,ecoscore_note,labels,code,ingredients,categories,countries
0,None,None,Issu pirate aop,z,999.0,[en:green-dot],2488753035032,None,None,fr
1,None,None,Southern fried chicken wrap,z,999.0,None,9310645338620,None,None,None
2,None,None,Strawberry some pulp,z,999.0,None,813622021809,None,None,us
3,None,None,junel fe,z,999.0,None,17198431,"ndc-0555-9026-58-junel-fe-1, 20, other-sexually-transmitted-diseases, ronly-shaping-women-s-health-л, norethindrone-acetate-ond-ethinyl-estradiol-tablets-us-and-ferrous-fumarate-tablets-each-light-yellow-tablet-contains-norethindrone-acetate, ethinyl-estradiol, 20-mcg, each-brown-tablet-contains-ferrous-fumarate, usp, each-tablet-dispenser-contains-21-light-yellow-tablets-and-7-brown-tablets-ferrous-fumarate-tablets-are-not-usp-for-dissolution-and-assay, 28-regimen-day-this-product, is-intended-prevent-pregnancy, it-does-not-protect-against-hiv-infecti-aids, like-all-oral-contraceptives",None,United States
4,None,Assiette,Salade japonaise,z,999.0,None,4902560170994,None,None,jp
5,None,None,Tisane de noel,z,999.0,None,3563210015730,None,None,None
6,Sugary snacks,None,Macaron XL,c,42.0,[en:no-gluten],4056489048855,None,"snacks, desserts, sweet-snacks, biscuits-and-cakes, pastries, macarons",be
7,None,None,Candy + Dispenser,z,999.0,None,810174011906,"dextrose, added-sugar, monosaccharide, glucose, e330, calcium-sterate, tapioca-dextrin, e1400, confectioners-glaze, e903, artificial-flavouring, flavouring, colour, e129, e102, e110, fd-c-bl, corn-syrup, disaccharide, e904",None,None
8,None,None,Fish oil,z,999.0,None,860000821131,None,None,us
10,None,None,Kosher Dill Spears,z,999.0,"[en:no-gluten, en:kosher]",886926216769,"cucumber, vegetable, fruit-vegetable, water, vinegar, salt, e508, e511, e509, e211, natural-flavouring, flavouring, e433, and-yellow-5, preservative",salted-snacks,None


In [29]:
df['ecoscore_groups'].tail(10)

988    z
989    z
990    z
991    z
992    z
995    z
996    z
997    z
998    z
999    z
Name: ecoscore_groups, dtype: object

In [30]:
nan_counts = df.isna().sum()

# Compter les listes vides
empty_list_counts = df.map(lambda x: x == []).sum()

# Afficher les résultats
print("nombre de NaN ou None par colonne :")
print(nan_counts)

nombre de NaN ou None par colonne :
groups             608
packaging          838
name                 0
ecoscore_groups      0
ecoscore_note        0
labels             569
code                 0
ingredients        695
categories         534
countries          412
dtype: int64


In [31]:
print("nombre de listes vides par colonne :")
print(empty_list_counts)

nombre de listes vides par colonne :
groups               0
packaging            0
name                 0
ecoscore_groups      0
ecoscore_note        0
labels             105
code                 0
ingredients          0
categories           0
countries            0
dtype: int64


In [32]:
df['labels'].head(50)

0                        [en:green-dot]
1                                  None
2                                  None
3                                  None
4                                  None
5                                  None
6                        [en:no-gluten]
7                                  None
8                                  None
10            [en:no-gluten, en:kosher]
12                                 None
13                                   []
14     [en:no-gmos, en:non-gmo-project]
15                  [en:no-added-sugar]
16                                 None
17                                 None
18      [en:no-gluten, en:no-additives]
19                                 None
20                       [en:green-dot]
21                                 None
22                                 None
23                             [en:pdo]
24                       [en:green-dot]
25     [en:no-gmos, en:non-gmo-project]
26                                 None


In [33]:
df['ingredients'].head(50)

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  None
1                                                                                                                                                                                                                                                                                                                                                                                                               